# Extended childcare entitlement

[Official government guidance](https://www.gov.uk/check-eligible-free-childcare-if-youre-working?step-by-step-nav=f517cd57-3c18-4bb9-aa8b-1b907e279bf9) | [Legislation](https://www.legislation.gov.uk/uksi/2017/1160/pdfs/uksiem_20171160_en.pdf)

## Program description

### Overview

The program provides free childcare hours based on the child's age:
* Children aged **9 months to 2 years** receive **15 hours** of free childcare per week
* Children aged **3 to 4 years** receive **30 hours** of free childcare per week

### Eligibility requirements

#### Work status
Eligible individuals must be either employed or about to start employment. This includes those on qualifying leave types: **sick**, **annual**, **shared parental**, **maternity**, **paternity**, or **adoption** leave.

Non-working partners may qualify if they receive: **incapacity benefit**, **severe disablement allowance**, **carer's allowance**, **limited capability for work benefit**, or **contribution-based employment and support allowance**.

#### Income requirements
Minimum earnings thresholds (per **3 months**) vary by age:

| Age group | Minimum earnings | Weekly equivalent |
|-----------|-----------------|-------------------|
| **21+** | **£2,380** | **£183** |
| **18-20** | **£1,788** | **£137** |
| **Under 18/Apprentices** | **£1,331** | **£102** |

These thresholds are based on national minimum wage calculations for **16 hours** per week.

Maximum income limit: Adjusted net income must not exceed **£100,000** per year.

### Additional rules

#### Income assessment
The program offers flexible income assessment:
* Combines **multiple income sources**
* Allows **annual income averaging**
* Gives **self-employed** individuals choice between total or self-employment income

Certain types of income are excluded from calculations: **dividends**, **interest**, **property income**, and **pension payments**.

## Implementation

The extended childcare entitlement program is modeled through several interconnected components:

### 1. Parameter definition
Located in [`parameters/gov/dfe/extended_childcare_entitlement/`](https://github.com/PolicyEngine/policyengine-uk/tree/20ed1a9d77a3307b3e2bc4a0986ec606ab7fead9/policyengine_uk/parameters/gov/dfe/extended_childcare_entitlement), key parameters include:

* Income requirements:
  * [`countable_sources.yaml`](https://github.com/PolicyEngine/policyengine-uk/tree/20ed1a9d77a3307b3e2bc4a0986ec606ab7fead9/policyengine_uk/parameters/gov/dfe/extended_childcare_entitlement): Defines eligible income sources
  * [`limit.yaml`](https://github.com/PolicyEngine/policyengine-uk/tree/20ed1a9d77a3307b3e2bc4a0986ec606ab7fead9/policyengine_uk/parameters/gov/dfe/extended_childcare_entitlement): Sets the £100,000 maximum income threshold
  * [`minimum_weekly_hours.yaml`](https://github.com/PolicyEngine/policyengine-uk/tree/20ed1a9d77a3307b3e2bc4a0986ec606ab7fead9/policyengine_uk/parameters/gov/dfe/extended_childcare_entitlement): Specifies required working hours

* Program specifics:
  * [`hours.yaml`](https://github.com/PolicyEngine/policyengine-uk/tree/20ed1a9d77a3307b3e2bc4a0986ec606ab7fead9/policyengine_uk/parameters/gov/dfe/extended_childcare_entitlement): Defines entitlement hours by age group
  * [`expense_rate.yaml`](https://github.com/PolicyEngine/policyengine-uk/tree/20ed1a9d77a3307b3e2bc4a0986ec606ab7fead9/policyengine_uk/parameters/gov/dfe/extended_childcare_entitlement): Sets childcare funding rates
  * [`weeks_per_year.yaml`](https://github.com/PolicyEngine/policyengine-uk/tree/20ed1a9d77a3307b3e2bc4a0986ec606ab7fead9/policyengine_uk/parameters/gov/dfe/extended_childcare_entitlement): Specifies annual coverage
  * [`disability_criteria.yaml`](https://github.com/PolicyEngine/policyengine-uk/blob/20ed1a9d77a3307b3e2bc4a0986ec606ab7fead9/policyengine_uk/parameters/gov/dfe/extended_childcare_entitlement/disability_criteria.yaml): Lists qualifying disability conditions

### 2. Income assessment
Implemented in [`extended_childcare_entitlement_meets_income_requirements.py`](https://github.com/PolicyEngine/policyengine-uk/tree/20ed1a9d77a3307b3e2bc4a0986ec606ab7fead9/policyengine_uk/variables/gov/dfe/extended_childcare_entitlement/conditions):
* Calculates total eligible income from specified sources
* Converts yearly income to quarterly periods
* Applies minimum wage thresholds based on work hours
* Checks against maximum income limit

Example table:

| Child age | Is compulsory school age | Weekly hours entitlement |
|-----------|-------------------------|------------------------|
| 1         | False                   | 15                     |
| 3         | False                   | 30                     |
| 4         | False                   | 30                     |
| 5         | True                    | 0                      |

### 3. Work condition verification
Handled by [`extended_childcare_entitlement_work_condition.py`](https://github.com/PolicyEngine/policyengine-uk/tree/20ed1a9d77a3307b3e2bc4a0986ec606ab7fead9/policyengine_uk/variables/gov/dfe/extended_childcare_entitlement/conditions):
* Verifies employment status
* Handles special cases for:
  * Single parents (must be working)
  * Couples (both working or one working with partner qualifying for disability benefits)
  * Carers (considers UC Carer Element)
* Includes disability-based eligibility checks using defined criteria

Example table:

| Adult age | Quarterly earnings | Is over £100k | Meets minimum earnings | Is income eligible |
|-----------|-------------------|---------------|----------------------|-------------------|
| 22        | £2,500            | False         | True                 | True              |
| 19        | £1,500            | False         | False                | False             |
| 35        | £10,000           | True          | True                 | False             |
| 18        | £1,400            | False         | True                 | True              |

### 4. Overall eligibility determination
Managed by [`extended_childcare_entitlement_eligible.py`](https://github.com/PolicyEngine/policyengine-uk/tree/20ed1a9d77a3307b3e2bc4a0986ec606ab7fead9/policyengine_uk/variables/gov/dfe/extended_childcare_entitlement):
* Combines results from income and work condition checks
* Applies eligibility rules at the benefit unit level
* Ensures all adult members (except children) meet requirements
* Returns final boolean eligibility determination

Example table:

| Family composition | Is partner 1 working | Is partner 2 working | Has qualifying disability | Meets work condition |
|-------------------|---------------------|---------------------|------------------------|-------------------|
| Single            | True                | N/A                 | N/A                    | True              |
| Couple            | True                | True                | False                  | True              |
| Couple            | True                | False               | True                   | True              |
| Couple            | False               | False               | True                   | False             |

### 5. Entitlement calculation
Final calculations in [`extended_childcare_entitlement.py`](https://github.com/PolicyEngine/policyengine-uk/tree/20ed1a9d77a3307b3e2bc4a0986ec606ab7fead9/policyengine_uk/variables/gov/dfe/extended_childcare_entitlement):
* Determines weekly hours based on child's age
* Applies appropriate funding rates
* Calculates total annual entitlement value
* Considers:
  * Child's age for hours allocation
  * Applicable expense rates
  * Number of weeks per year
  * Multiple children in the benefit unit

Example table:

| Child age | Weekly hours | Funding rate | Annual entitlement |
|-----------|-------------|-------------|-------------------|
| 2         | 15          | £8.28       | £4,719.6         |
| 3         | 30          | £5.88     | £6,703.2         |